In [ ]:
#resnet-metabolite-0.2test

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

# ------------------ Step 1 ------------------

df = pd.read_csv("lasso_metabolite.csv")  
X = df.iloc[:, 3:]  
y = df.iloc[:, 1]     


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ------------------ Step 2 ------------------

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=6)#0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)


# -----------------  Step 3  -----------------
class ResNet(nn.Module):
    def __init__(self, input_dim):
        super(ResNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32)
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.fc(out)
        return out  # raw logits, use BCEWithLogitsLoss

# ----------------- Step 4 -----------------
input_dim = X_train_tensor.shape[1]
model = ResNet(input_dim).to(X_train_tensor.device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------- Step 5 -----------------
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    #loss = criterion(outputs.squeeze(), y_train_tensor[:, 1])  # 只用 SRFS 这一列
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# -----------------  Step 6 -----------------
def evaluate_model(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        # training set
        train_logits = model(X_train_tensor).squeeze()
        train_probs = torch.sigmoid(train_logits).cpu().numpy()
        train_preds = (train_probs > 0.5).astype(int)
        y_train_np = y_train_tensor.cpu().numpy()
        auc_train = roc_auc_score(y_train_np, train_probs)
        acc_train = accuracy_score(y_train_np, train_preds)
        cm_train = confusion_matrix(y_train_np, train_preds)

        # test set
        test_logits = model(X_test_tensor).squeeze()
        test_probs = torch.sigmoid(test_logits).cpu().numpy()
        test_preds = (test_probs > 0.5).astype(int)
        y_test_np = y_test_tensor.cpu().numpy()
        auc_test = roc_auc_score(y_test_np, test_probs)
        acc_test = accuracy_score(y_test_np, test_preds)
        cm_test = confusion_matrix(y_test_np, test_preds)

    
    print("\n[训练集评估]")
    print(f"AUC: {auc_train:.4f}")
    print(f"Accuracy: {acc_train:.4f}")
    print("Confusion Matrix:")
    print(cm_train)

    print("\n[测试集评估]")
    print(f"AUC: {auc_test:.4f}")
    print(f"Accuracy: {acc_test:.4f}")
    print("Confusion Matrix:")
    print(cm_test)

    return auc_train, auc_test, acc_train, acc_test
    
# ----------------- Step 7 -----------------
evaluate_model(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)